In [ ]:
%pip install -i https://pypi.gurobi.com gurobipy

Looking in indexes: https://pypi.gurobi.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 26.8 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import numpy as np
import math

# STATIC

In [ ]:
def vrp_static_2(d, n, K, Q, T):
  b = []
  for i in range(n+K):
    b.append(int(i < n))

  model = gp.Model("Travelling Salesman Problem VRP Index 2")

  # Create variables
  x = model.addVars(n+K, n+K, vtype=GRB.BINARY, name="x")
  y = model.addVars(n+K, n+K, name="y")
  z = model.addVars(n+K, name="z")

  # Ensure tour
  model.addConstrs((quicksum(x[city_1, city_2] for city_2 in range(n+K)) == 1
                      for city_1 in range(n+K)), name="arrival")
  model.addConstrs((quicksum(x[city_1, city_2] for city_1 in range(n+K)) == 1
                      for city_2 in range(n+K)), name="departure")

  """
  # Total number of tours
  model.addConstr(quicksum(x[city_1, 0] for city_1 in range(n+1)) ==
                  quicksum(x[0, city_2] for city_2 in range(n+1)))
  model.addConstr(quicksum(x[0, city_2] for city_2 in range(n+1)) <= K)
  """

  # Demand at location j
  model.addConstrs(quicksum(y[city_1, city_2]-y[city_2, city_1] for city_1 in range(n+K)) == b[city_2]
                   for city_2 in range(n))

  # Total demand
  model.addConstr(quicksum(y[city_1, city_2] for city_1 in range(n, n+K) for city_2 in range(n+K)) ==
                  quicksum(b[city_2] for city_2 in range(n)))

  # Transportation capacity
  model.addConstrs((b[city_2]*x[city_1, city_2] <= y[city_1, city_2])
                    for city_1 in range(n+K)
                    for city_2 in range(n+K))
  model.addConstrs((y[city_1, city_2] <= (Q-b[city_1])*x[city_1, city_2])
                    for city_1 in range(n+K)
                    for city_2 in range(n+K))

  # Empty vehicle at the end
  model.addConstrs((y[city_2, city_1] == 0) for city_1 in range(n, n+K) for city_2 in range(n+K))

  # Subtourelimination constraints
  for city_1 in range(n, n+K):
      for city_2 in range(n, n+K):
          if city_1 != city_2:
              model.addConstr(z[city_1] - z[city_2] + K * x[city_1, city_2] <= K - 1)
          else:
              model.addConstr(x[city_1, city_1] == 0)

  # Set objective
  model.setObjective(quicksum(d[city_1][city_2] * x[city_1, city_2]
                    for city_1 in range(n+K)
                    for city_2 in range(n+K)),
                     GRB.MINIMIZE)

  # Run optimization
  model.optimize()

  # Get solution
  print("x")
  solution_x = model.getAttr('x', x)
  for city_1 in range(n+K):
      for city_2 in range(n+K):
          if solution_x[city_1, city_2] > 1e-5:
              print('%s -> %s: %g' % (city_1, city_2, solution_x[city_1, city_2]))
  print("y")
  solution_y = model.getAttr('x', y)
  for city_1 in range(n+K):
      for city_2 in range(n+K):
          if solution_y[city_1, city_2] > 1e-5:
              print('%s -> %s: %g' % (city_1, city_2, solution_y[city_1, city_2]))
  print("Objective: "+str(model.objVal))

In [ ]:
K = 3

In [ ]:
BIG_COST = 1000000
coordinates = [[4, 1], [1, 2], [0, 5], [-3, 3], [-2, 1], [-5, 1], [-5, -1], [-1, -3], [3, -2], [6, -1]]
n = len(coordinates)
for _ in range(K):
  coordinates.append([0, 0])
num_cities = len(coordinates)
d = []
for i, source in enumerate(coordinates):
  d_src = []
  for j, sink in enumerate(coordinates):
    if i == j:
      d_src.append(-BIG_COST)
    else:
      d_x = source[0]-sink[0]
      d_y = source[1]-sink[1]
      d_src.append(math.sqrt(d_x*d_x+d_y*d_y))
  d.append(d_src)
  print(d_src)

[-1000000, 3.1622776601683795, 5.656854249492381, 7.280109889280518, 6.0, 9.0, 9.219544457292887, 6.4031242374328485, 3.1622776601683795, 2.8284271247461903, 4.123105625617661, 4.123105625617661, 4.123105625617661]
[3.1622776601683795, -1000000, 3.1622776601683795, 4.123105625617661, 3.1622776601683795, 6.082762530298219, 6.708203932499369, 5.385164807134504, 4.47213595499958, 5.830951894845301, 2.23606797749979, 2.23606797749979, 2.23606797749979]
[5.656854249492381, 3.1622776601683795, -1000000, 3.605551275463989, 4.47213595499958, 6.4031242374328485, 7.810249675906654, 8.06225774829855, 7.615773105863909, 8.48528137423857, 5.0, 5.0, 5.0]
[7.280109889280518, 4.123105625617661, 3.605551275463989, -1000000, 2.23606797749979, 2.8284271247461903, 4.47213595499958, 6.324555320336759, 7.810249675906654, 9.848857801796104, 4.242640687119285, 4.242640687119285, 4.242640687119285]
[6.0, 3.1622776601683795, 4.47213595499958, 2.23606797749979, -1000000, 3.0, 3.605551275463989, 4.123105625617661

In [ ]:
vrp_static_2(d=d, n=n, K=K, Q=100, T=16)

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 423 rows, 351 columns and 1314 nonzeros
Model fingerprint: 0x2063c545
Variable types: 182 continuous, 169 integer (169 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 140 rows and 72 columns
Presolve time: 0.02s
Presolved: 283 rows, 279 columns, 1050 nonzeros
Variable types: 123 continuous, 156 integer (156 binary)
Found heuristic solution: objective 72.3702525
Found heuristic solution: objective 66.5726673

Root relaxation: objective 3.040379e+01, 206 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds  

In [ ]:
BIG_COST = 1000000
coordinates = [[i+1, i+1] for i in range(3)]
n = len(coordinates)
for _ in range(K):
  coordinates.append([0, 0])
num_cities = len(coordinates)
d = []
for i, source in enumerate(coordinates):
  d_src = []
  for j, sink in enumerate(coordinates):
    if i == j:
      d_src.append(-BIG_COST)
    else:
      d_x = source[0]-sink[0]
      d_y = source[1]-sink[1]
      d_src.append(math.sqrt(d_x*d_x+d_y*d_y))
  d.append(d_src)
  print(d_src)

[-1000000, 1.4142135623730951, 2.8284271247461903, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951]
[1.4142135623730951, -1000000, 1.4142135623730951, 2.8284271247461903, 2.8284271247461903, 2.8284271247461903]
[2.8284271247461903, 1.4142135623730951, -1000000, 4.242640687119285, 4.242640687119285, 4.242640687119285]
[1.4142135623730951, 2.8284271247461903, 4.242640687119285, -1000000, 0.0, 0.0]
[1.4142135623730951, 2.8284271247461903, 4.242640687119285, 0.0, -1000000, 0.0]
[1.4142135623730951, 2.8284271247461903, 4.242640687119285, 0.0, 0.0, -1000000]


In [ ]:
vrp_static_2(d=d, n=n, K=K, Q=100, T=16)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 115 rows, 78 columns and 285 nonzeros
Model fingerprint: 0x54620740
Variable types: 42 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 11.3137085
Presolve removed 63 rows and 30 columns
Presolve time: 0.00s
Presolved: 52 rows, 48 columns, 168 nonzeros
Variable types: 18 continuous, 30 integer (30 binary)

Root relaxation: objective 5.713994e+00, 33 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.71399    0    6   1

# DYNAMIC

In [ ]:
def vrp_dynamic_2(d, n, K, Q, T, UNITS = 20 , BIG_COST = 1000000):
  H = len(d)
  b = []
  for i in range(n+K):
    b.append(int(i < n))

  model = gp.Model("Travelling Salesman Problem VRP Index 2")

  # Create variables
  x = model.addVars(n+K, n+K, vtype=GRB.BINARY, name="x")
  y = model.addVars(n+K, n+K, name="y")
  z = model.addVars(n+K, name="z")
  t = model.addVars(n+K, name="t")
  h = model.addVars(n+K, H, vtype=GRB.BINARY, name="h")
  xh = model.addVars(n+K, n+K, H, vtype=GRB.BINARY, name="xh")
  vrp_duration = model.addVar(name="vrp_duration")

  # Ensure tour
  model.addConstrs((quicksum(x[city_1, city_2] for city_2 in range(n+K)) == 1
                      for city_1 in range(n+K)), name="arrival")
  model.addConstrs((quicksum(x[city_1, city_2] for city_1 in range(n+K)) == 1
                      for city_2 in range(n+K)), name="departure")

  """
  # Total number of tours
  model.addConstr(quicksum(x[city_1, 0] for city_1 in range(n+1)) ==
                  quicksum(x[0, city_2] for city_2 in range(n+1)))
  model.addConstr(quicksum(x[0, city_2] for city_2 in range(n+1)) <= K)
  """

  # Demand at location j
  model.addConstrs(quicksum(y[city_1, city_2]-y[city_2, city_1] for city_1 in range(n+K)) == b[city_2]
                   for city_2 in range(n))

  # Total demand
  model.addConstr(quicksum(y[city_1, city_2] for city_1 in range(n, n+K) for city_2 in range(n+K)) ==
                  quicksum(b[city_2] for city_2 in range(n)))

  # Transportation capacity
  model.addConstrs((b[city_2]*x[city_1, city_2] <= y[city_1, city_2])
                    for city_1 in range(n+K)
                    for city_2 in range(n+K))
  model.addConstrs((y[city_1, city_2] <= (Q-b[city_1])*x[city_1, city_2])
                    for city_1 in range(n+K)
                    for city_2 in range(n+K))

  # Empty vehicle at the end
  model.addConstrs((y[city_2, city_1] == 0) for city_1 in range(n, n+K) for city_2 in range(n+K))

  # Subtourelimination constraints
  for city_1 in range(n, n+K):
      for city_2 in range(n, n+K):
          if city_1 != city_2:
              model.addConstr(z[city_1] - z[city_2] + K * x[city_1, city_2] <= K - 1)
          else:
              model.addConstr(x[city_1, city_1] == 0)

  # Hour constraints
  model.addConstrs((h[city, hour]*UNITS*hour <= t[city]) for city in range(n+K) for hour in range(H))
  model.addConstrs((h[city, hour]*UNITS*(hour+1)-1e-9 + (1-h[city, hour])*BIG_COST >= t[city]) for city in range(n+K) for hour in range(H))
  model.addConstrs(quicksum(h[city, hour] for hour in range(H)) == 1 for city in range(n+K))

  # x & h constraints
  model.addConstrs((x[city_1, city_2] * h[city_1, hour] == xh[city_1, city_2, hour])
                   for city_1 in range(n+K) for city_2 in range(n+K) for hour in range(H))

  # Time constraints
  model.addConstr(t[n] == 0)
  for city_2 in range(n+K):
      if city_2 != n:
          model.addConstr(quicksum(
              (xh[city_1, city_2, hour] * (t[city_1] + d[hour][city_1][city_2])) for city_1 in range(n+K) for hour in range(H)) == t[city_2])

  # Final distance
  model.addConstrs((xh[city_1, n, hour] * (t[city_1] + d[hour][city_1][n]) <= vrp_duration)
                    for city_1 in range(n+K) for hour in range(H))
  model.addConstr(vrp_duration <= H*UNITS)

  # Set objective
  model.setObjective(vrp_duration, GRB.MINIMIZE)

  # Run optimization
  model.optimize()

  # Get solution
  print("x")
  solution_x = model.getAttr('x', x)
  for city_1 in range(n+K):
      for city_2 in range(n+K):
          if solution_x[city_1, city_2] == 1:
              print('%s -> %s: %g' % (city_1, city_2, solution_x[city_1, city_2]))
  print("y")
  solution_y = model.getAttr('x', y)
  for city_1 in range(n+K):
      for city_2 in range(n+K):
          if solution_y[city_1, city_2] > 1e-5:
              print('%s -> %s: %g' % (city_1, city_2, solution_y[city_1, city_2]))
  print("t")
  solution_t = model.getAttr('x', t)
  for city in range(n+K):
    print('%s: %g' % (city, solution_t[city]))
  print("h")
  solution_h = model.getAttr('x', h)
  for city in range(n+K):
      for hour in range(H):
          if solution_h[city_1, hour] == 1:
              print('%s: %s' % (city, hour))
  print("Objective: "+str(model.objVal))

In [ ]:
d_dynamic = []
for hour in range(2):
  d_new = []
  for d_src in d:
    d_src_new = []
    for val in d_src:
      d_src_new.append(val*(hour+1))
    d_new.append(d_src_new)
  d_dynamic.append(d_new)

In [ ]:
for row in d_dynamic[0]:
  print(row)

[-1000000, 1.4142135623730951, 2.8284271247461903, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951]
[1.4142135623730951, -1000000, 1.4142135623730951, 2.8284271247461903, 2.8284271247461903, 2.8284271247461903]
[2.8284271247461903, 1.4142135623730951, -1000000, 4.242640687119285, 4.242640687119285, 4.242640687119285]
[1.4142135623730951, 2.8284271247461903, 4.242640687119285, -1000000, 0.0, 0.0]
[1.4142135623730951, 2.8284271247461903, 4.242640687119285, 0.0, -1000000, 0.0]
[1.4142135623730951, 2.8284271247461903, 4.242640687119285, 0.0, 0.0, -1000000]


In [ ]:
for row in d_dynamic[1]:
  print(row)

[-2000000, 2.8284271247461903, 5.656854249492381, 2.8284271247461903, 2.8284271247461903, 2.8284271247461903]
[2.8284271247461903, -2000000, 2.8284271247461903, 5.656854249492381, 5.656854249492381, 5.656854249492381]
[5.656854249492381, 2.8284271247461903, -2000000, 8.48528137423857, 8.48528137423857, 8.48528137423857]
[2.8284271247461903, 5.656854249492381, 8.48528137423857, -2000000, 0.0, 0.0]
[2.8284271247461903, 5.656854249492381, 8.48528137423857, 0.0, -2000000, 0.0]
[2.8284271247461903, 5.656854249492381, 8.48528137423857, 0.0, 0.0, -2000000]


In [ ]:
vrp_dynamic_2(d=d_dynamic, n=n, K=K, Q=100, T=16)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 147 rows, 169 columns and 341 nonzeros
Model fingerprint: 0xc3b92be0
Model has 89 quadratic constraints
Variable types: 49 continuous, 120 integer (120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 4 rows and 60 columns
Presolve time: 0.00s
Presolved: 376 rows, 178 columns, 990 nonzeros
Variable types: 68 continuous, 110 integer (110 binary)
Found heuristic solution: objective 11.3137085

Root relaxation: objective 0.000000e+00, 99 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 